In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
dataset=pd.read_csv("dataset.csv",header = 0)
dataset.head()

,name,npm,class
0,Diaz Ilyasa Azrurrafi Saiful,1806200154,Despro-1
1,Dio Fajrie Fadlullah,1806200324,Despro-1
2,Muhammad Miftah Faridh,1806148782,Despro-2
3,Abdul Fattah Ismail,1806200255,Despro-2
4,Helmi Arrazy,1806200141,Despro-1


In [3]:
from fuzzywuzzy import fuzz

def fuzzScore(dataset, namecol, datainput):
    dataset['scoreRatio'] = 0
    dataset['scorePartial'] = 0
    dataset['scoreSet'] = 0
    dataset['scoreSort'] = 0
    for index, rows in dataset.iterrows():
        data = rows[namecol]
        dataset.loc[index, 'scorePartial'] = fuzz.partial_ratio(data, datainput)
        dataset.loc[index, 'scoreSort'] = fuzz.token_sort_ratio(data, datainput)
        dataset.loc[index, 'scoreSet'] = fuzz.token_set_ratio(data, datainput)
        dataset.loc[index, 'scoreRatio'] = fuzz.ratio(data, datainput)
    return dataset

In [4]:
def getFullname(dataset, namecol, datainput):
    check = fuzzScore(dataset, namecol, datainput)
    tester = {'type': ['scoreRatio', 'scoreSet', 'scoreSort', 'scorePartial'],
        'score':0,
        'values':0
        }
    df = pd.DataFrame(tester, columns = ['type', 'score','values'])
    for index, test in df.iterrows():
        check.sort_values(by = test.type, ascending=False, ignore_index=True, inplace=True)
        df.loc[index, 'score'] =check[test.type][0]
        df.loc[index, 'values'] = str(check.head()[namecol].tolist())
    df.sort_values(by = 'score', ascending=False, ignore_index=True, inplace=True)
    fullName = df['values'][0].replace('\'','')[1:-1].split(',')[0]
    return fullName, df

In [5]:
fullname, val = getFullname(dataset, 'name', 'alfian')
print(fullname)
print(val)

Alfian Badrul Isnan
           type  score                                             values
0      scoreSet    100  ['Alfian Badrul Isnan', 'Arvalinno', 'Aurellio...
1  scorePartial     83  ['Alfian Badrul Isnan', 'Arvalinno', 'Ferdian ...
2    scoreRatio     53  ['Arvalinno', 'Ferdian Julianto', 'Natalia Kri...
3     scoreSort     53  ['Arvalinno', 'Aurellio FIshandy', 'Alvin Audi...


In [6]:
def dataFillFullName(dataset, namecol, inputdata, nicknamecol, fullnamecol):
    length = len(inputdata)
    for index, rows in inputdata.iterrows():
        out, var = getFullname(dataset, namecol, rows[nicknamecol])
        inputdata.loc[index, fullnamecol] = out
        print('Check : ' + str(index + 1) + '/' + str(length) + ' | ' + rows[nicknamecol] + ' ~ ' + out)
    return inputdata

In [7]:
inputData=pd.read_csv("input.csv", header=0)
inputData.head()

,NAMA ACAK,NAMA LENGKAP
0,diaz,NaN
1,dio,NaN
2,miptah,NaN
3,abdol,NaN
4,helmy,NaN


In [8]:
outputData = dataFillFullName(dataset, 'name', inputData, 'NAMA ACAK', 'NAMA LENGKAP')
outputData.head()

Check : 1/67 | diaz ~ Diaz Ilyasa Azrurrafi Saiful
Check : 2/67 | dio ~ Dio Fajrie Fadlullah
Check : 3/67 | miptah ~ Muhammad Miftah Faridh
Check : 4/67 | abdol ~ Fadly Ahmad Firdausy
Check : 5/67 | helmy ~ Sherly
Check : 6/67 | martin ~ Martin Hizkia Parasi
Check : 7/67 | hanfey ~ Aurellio FIshandy
Check : 8/67 | arva ~ Arvalinno
Check : 9/67 | brian ~ Ferdian Julianto
Check : 10/67 | pk ~ Ricky
Check : 11/67 | arif ~ Syarif Hidayat
Check : 12/67 | kepin ~ Kevin Darmawan
Check : 13/67 | alfi ~ Muhammad Alfi Aldolio
Check : 14/67 | farhan ~ Muhammad Farhan Almasyhur
Check : 15/67 | salma ~ Mohammad Salman Alfarisi
Check : 16/67 | alfi ~ Muhammad Alfi Aldolio
Check : 17/67 | ricky ~ Ricky
Check : 18/67 | zidan ~ Muhammad Zidan Arsyad
Check : 19/67 | fadli ~ Arvalinno
Check : 20/67 | sadam ~ Muhammad Saddam
Check : 21/67 | ferdi ~ Ferdian Julianto
Check : 22/67 | jona ~ Jonathan
Check : 23/67 | nat ~ Jonathan
Check : 24/67 | kisas ~ Qisas T Hasanudin
Check : 25/67 | vinsen ~ Arvalinno
Ch

,NAMA ACAK,NAMA LENGKAP
0,diaz,Diaz Ilyasa Azrurrafi Saiful
1,dio,Dio Fajrie Fadlullah
2,miptah,Muhammad Miftah Faridh
3,abdol,Fadly Ahmad Firdausy
4,helmy,Sherly


In [9]:
datatest=pd.read_csv("datatest.csv",header = 0)
datatest.head()

,name,test1,test2,test3,test4,test5
0,Diaz Ilyasa Azrurrafi Saiful,diaz,dias,diyaz,diyas,saeful
1,Dio Fajrie Fadlullah,dio,deo,diyo,fajri,fadlul
2,Muhammad Miftah Faridh,miptah,farid,farit,miptah farid,parid
3,Abdul Fattah Ismail,abdol,pattah,fatah,fatta,patta
4,Helmi Arrazy,helmy,halmi,helm,elmi,arasi


In [10]:
def dataFillCheckError(datatest, namecol, testcol, dataoutput, nicknamecol, fullnamecol):
    length = len(dataoutput)
    passed = 0
    for index, output in dataoutput.iterrows():
        scan = datatest.loc[datatest[testcol] == output[nicknamecol]][namecol][index]
        if(scan == output[fullnamecol]):
            passed+=1
            print('PASS ' + str(passed) + '/' + str(length))
        else:
            print('----------------------------------')
            print('FAILED on : ' + str(index))
            print('NickName  : ' + output[nicknamecol])
            print('Output    : ' + output[fullnamecol])
            print('Original  : ' + scan)
            print('----------------------------------')
    print('\n\n'
          'Passed : {:.2f}%'.format(passed/length*100))
    print('Error  : {:.2f}%'.format((length-passed)/length*100))

In [11]:
dataFillCheckError(datatest, 'name', 'test1', outputData, 'NAMA ACAK', 'NAMA LENGKAP')

PASS 1/67
PASS 2/67
PASS 3/67
----------------------------------
FAILED on : 3
NickName  : abdol
Output    : Fadly Ahmad Firdausy
Original  : Abdul Fattah Ismail
----------------------------------
----------------------------------
FAILED on : 4
NickName  : helmy
Output    : Sherly
Original  : Helmi Arrazy
----------------------------------
PASS 4/67
----------------------------------
FAILED on : 6
NickName  : hanfey
Output    : Aurellio FIshandy
Original  : Hanvey Xavero
----------------------------------
PASS 5/67
----------------------------------
FAILED on : 8
NickName  : brian
Output    : Ferdian Julianto
Original  : Bryan Oliver
----------------------------------
----------------------------------
FAILED on : 9
NickName  : pk
Output    : Ricky
Original  : Philipus Kristian Renaldy
----------------------------------
----------------------------------
FAILED on : 10
NickName  : arif
Output    : Syarif Hidayat
Original  : Arief Saferman
----------------------------------
PASS 6/67
P

In [12]:
outputData.to_csv(r'export\output.csv', index = False)